# Planet Download Example
This notebook shows how to download planet imagery uploaded to a private s3 bucket for OPERA DSWx validation

In [1]:
import geopandas as gpd
import boto3

### AWS Credentials
In order to download imagery from the private bucket, JPL RSA access and OPERA Calval AWS credentials are needed

In [2]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

### S3 Buckets
A table containing image metadata is located on a public s3 bucket, but the commercial image files can't be openly shared and are stored in a seperate private bucket. S3 keys (path within a bucket) that point to the image file locations are stored in the metadata table in the public bucket

In [3]:
bucket_name = 'opera-calval-database-dswx'
private_bucket_name = 'opera-calval-database-dswx-private'

### Read Image metadata table
Image metadata is stored in a geojson table, which can be read as a geodataframe using geopandas. This dataframe can be spatial searched or searched by metadata fields

In [4]:
obj = s3.Object(bucket_name,'image.geojson')
image_table_data = obj.get()['Body']
images = gpd.read_file(image_table_data)
images.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
0,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
1,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
2,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
3,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
4,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None


### Download Function
This function takes a single row of the image geodataframe and a download location (folder path on local computer) and downloads the image file associated with the provided row

In [5]:
def downloadImage(row,downloadDir):
    bucket = row.s3_bucket.iloc[0]
    key = row.s3_key_image.iloc[0]
    image_name = row.image_name.iloc[0]
    response = s3_client.download_file(bucket,key,downloadDir+image_name)

### Basic Usage Example
This example extracts the first image from the image table and downloads it to a sub directory within the current working directory called "downloads".

In [6]:
downloadDir = './downloads/'

In [10]:
search = images[images.site_name == '4_8']
search.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
87,0.0,HLS.S30.T47ULQ.2021254T043701.v2.0,2021-09-12T03:40:49,20210912_034049_22_2421,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_8/20210912_034049_22_2421/20210912_0340...,97.608045,4_8,0.955574,"POLYGON ((97.44674 49.52974, 97.37427 49.34386..."


In [11]:
row = search.iloc[[0]]

In [12]:
downloadImage(row,downloadDir)

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found